In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Object Detection using multiple class dataset

Cold Drinks Inventory Dataset

Test Demo Version

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

import pandas as pd
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
# %matplotlib inline

In [ ]:
import yaml

dataset_config = {
    'train': '/kaggle/input/cold-drinks-inventory-dataset/Finalize/images/train',  # Adjust this path
    'val': '/kaggle/input/cold-drinks-inventory-dataset/Finalize/images/test',  # Adjust this path if you have a separate validation set
    'nc': 6,
    'names': ['Coca Cola',
             'Sprite',
             'Pepsi',
             'Mountain Dew',
             '7UP',
             'Fanta']  # Adjust this list if you have more classes
}

# Specify the path where you want to save the YAML file
yaml_path = '/kaggle/working/dataset.yaml'

# Write the dictionary to a YAML file
with open(yaml_path, 'w') as yamlfile:
    yaml.dump(dataset_config, yamlfile, default_flow_style=False)

In [ ]:
# !git clone https://github.com/ultralytics/yolov5  # Clone YOLOv5 repository
# %cd yolov5
# !pip install -r requirements.txt  # Install required dependencies

In [ ]:
# !python train.py --img 640 --batch 16 --epochs 100 --data /kaggle/working/dataset.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name yolo_custom

In [ ]:
# pip install torch torchvision

In [ ]:
import torch
import cv2
import matplotlib.pyplot as plt
import numpy as np

def detect_and_visualize(model_path, image_path, conf_threshold=0.3, iou_threshold=0.45):
    # Load the model
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=True)
    
    # Set confidence and IoU thresholds
    model.conf = conf_threshold
    model.iou = iou_threshold
    
    # Perform inference
    results = model(image_path)
    
    # Process results to extract labels and bounding box coordinates
    labels, cords = results.xyxyn[0][:, -1].cpu().numpy(), results.xyxyn[0][:, :-1].cpu().numpy()
    
    # Load image using OpenCV and convert it to RGB format
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Dictionary to store counts of each label
    label_counts = {}
    
    # Draw bounding boxes and labels on the image
    for cord in cords:
        # Ensure bbox contains x1, y1, x2, y2, conf
        if len(cord) == 5:
            x1, y1, x2, y2, conf = cord
            if conf >= conf_threshold:
                start = (int(x1 * img.shape[1]), int(y1 * img.shape[0]))
                end = (int(x2 * img.shape[1]), int(y2 * img.shape[0]))
                cv2.rectangle(img, start, end, (255, 0, 0), 2)
                label = int(labels[np.where(cords[:, :4] == cord[:4])[0][0]])  # Extract label
                cv2.putText(img, f'{label} {conf:.2f}', start, cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
                # Update label count
                label_counts[label] = label_counts.get(label, 0) + 1
    
    # Visualization using matplotlib
    plt.figure(figsize=(20, 10))  # Adjusted for better fit on standard screens
    plt.imshow(img)
    plt.axis('off')  # Hides the axis to focus on the image
    plt.show()

    # Print the count of detected objects
    print(f'Detected objects: {len(cords)}')
    
    # Print count of each label
    for label, count in label_counts.items():
        print(f'Label {label}: {count}')

In [ ]:
# Example usage
%matplotlib inline
model_path = '/kaggle/input/previous-version-output/yolov5/runs/train/yolo_custom/weights/best.pt'  # Update this path
image_path = '/kaggle/input/testdrinks/image_2024_02_23T12_06_03_052Z.png'
detect_and_visualize(model_path, image_path)

In [ ]:
image_path = '/kaggle/input/cold-drinks-inventory-dataset/Finalize/images/train/0bbc8618-225e-4b2a-9fe1-571af5938737.jpg'

detect_and_visualize(model_path, image_path)

In [ ]:
image_path = '/kaggle/input/testdrinks/image_2024_02_23T12_12_26_816Z.png'

detect_and_visualize(model_path, image_path)

In [ ]:
image_path = '/kaggle/input/testdrinks/image_2024_02_23T12_11_59_469Z.png'

detect_and_visualize(model_path, image_path)

In [ ]:
image_path = '/kaggle/input/testdrinks/im_two.jpg'

detect_and_visualize(model_path, image_path)